## Data Cleaning and Entity Resolution
For this session, we process two raw datasets from Kaggle, cleaning and organizing them into five distict tables for integration into our relational database to support future querying. The ER diagram is shown as below:

In [ ]:
import pandas as pd
import numpy as np
import random

In [2]:
song_path = "/Users/hangsmacbookair/Desktop/CPSC 537/cpsc537proj/spotify-2023 2.csv"
song_raw = pd.read_csv(song_path, encoding = 'latin1')
song_raw.head()

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6


In [3]:
user_path = "/Users/hangsmacbookair/Desktop/CPSC 537/cpsc537proj/Spotify_data.xlsx"
user_raw = pd.read_excel(user_path)
user_raw.head()

,Age,Gender,spotify_usage_period,spotify_listening_device,spotify_subscription_plan,premium_sub_willingness,preffered_premium_plan,preferred_listening_content,fav_music_genre,music_time_slot,music_Influencial_mood,music_lis_frequency,music_expl_method,music_recc_rating,pod_lis_frequency,fav_pod_genre,preffered_pod_format,pod_host_preference,preffered_pod_duration,pod_variety_satisfaction
0,20-35,Female,More than 2 years,Smart speakers or voice assistants,Free (ad-supported),Yes,Family Plan-Rs 179/month,Podcast,Melody,Night,Sadness or melancholy,leisure time,Playlists,3,Daily,Comedy,Interview,Both,Both,Ok
1,12-20,Male,More than 2 years,Computer or laptop,Free (ad-supported),Yes,Individual Plan- Rs 119/ month,Podcast,Rap,Afternoon,Social gatherings or parties,Workout session,Playlists,2,Several times a week,Comedy,Interview,Both,None,Satisfied
2,35-60,Others,6 months to 1 year,Smart speakers or voice assistants,Free (ad-supported),Yes,Student Plan-Rs 59/month,Podcast,Pop,Night,Relaxation and stress relief,"Study Hours, While Traveling",Playlists,4,Once a week,Sports,Interview,None,Both,Satisfied
3,20-35,Female,1 year to 2 years,"Smartphone, Smart speakers or voice assistants",Free (ad-supported),No,None,Music,Melody,Night,"Relaxation and stress relief, Social gathering...","Office hours, Workout session, leisure time","recommendations, Playlists",4,Never,None,None,None,None,Ok
4,20-35,Female,1 year to 2 years,Smartphone,Free (ad-supported),No,None,Music,Melody,Night,Relaxation and stress relief,leisure time,"recommendations, Playlists",4,Rarely,Lifestyle and Health,Story telling,Well known individuals,Both,Ok


### song table:

In [6]:
# Drop NaNs from song raw, 817 rows left
song_df = song_raw.dropna()
# Create a column of release date
song_df = song_df.rename(columns = {"released_year" : "year", "released_month" : "month", "released_day" : "day"})
song_df["released_date"] = pd.to_datetime(song_df[["year", "month", "day"]])
# Create a column of song_id 000-817
song_df["song_id"] = [str(i).zfill(len(str(len(song_df)))) for i in range(len(song_df))]
song_df = song_df.drop(["year", "month", "day"], axis = 1)
song_df = song_df.drop(["artist_count", "in_spotify_playlists", 
                        "in_spotify_charts", "in_apple_playlists", 
                        "in_apple_charts", "in_deezer_playlists", 
                        "in_deezer_charts", "in_shazam_charts", "mode"], axis = 1)
# Create a column of promotion, half songs are promoted
song_df["promotion"] = random.choices([0, 1], k = len(song_df), weights = [0.25, 0.75])
song_df.head()

,track_name,artist(s)_name,streams,bpm,key,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%,released_date,song_id,promotion
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",141381703,125,B,80,89,83,31,0,8,4,2023-07-14,000,1
1,LALA,Myke Towers,133716286,92,C#,71,61,74,7,0,10,4,2023-03-23,001,1
2,vampire,Olivia Rodrigo,140003974,138,F,51,32,53,17,0,31,6,2023-06-30,002,1
3,Cruel Summer,Taylor Swift,800840817,170,A,55,58,72,11,0,11,15,2019-08-23,003,1
4,WHERE SHE GOES,Bad Bunny,303236322,144,A,65,23,80,14,63,11,6,2023-05-18,004,1


### artist table:

In [7]:
artist_df = pd.DataFrame()
# Create artist_name column for artist
artist_df["artist_name"] = song_df["artist(s)_name"].unique()
# For rows that contains more than 1 names, separate them to multiple rows
artist_df["artist_name"] = artist_df["artist_name"].str.split(", ")
artist_df = artist_df.explode("artist_name")
# Strip useless spaces
artist_df["artist_name"] = artist_df["artist_name"].str.strip()
# Delete duplicate artist names
artist_df = artist_df.drop_duplicates()
# Create a column of artist_id 000-628
artist_df["artist_id"] = [str(i).zfill(len(str(len(artist_df)))) for i in range(len(artist_df))]
artist_df.head()

,artist_name,artist_id
0,Latto,000
0,Jung Kook,001
1,Myke Towers,002
2,Olivia Rodrigo,003
3,Taylor Swift,004


### song_artist table:

In [8]:
# Create song_artist table
song_artist_df = song_df.loc[:, ["song_id", "artist(s)_name"]]
# For songs with multiple artists, separate them to multiple rows
song_artist_df["artist(s)_name"] = song_artist_df["artist(s)_name"].str.split(", ")
song_artist_df = song_artist_df.explode("artist(s)_name")
song_artist_df = song_artist_df.drop_duplicates()
# Left join artist table
song_artists_df = pd.merge(song_artist_df, artist_df, left_on = "artist(s)_name", right_on = "artist_name", how = "left")
song_artists_df = song_artists_df.loc[:, ["song_id", "artist_id"]].drop_duplicates()
song_artists_df.head()
#song_artists_df.shape

,song_id,artist_id
0,000,000
1,000,001
2,001,002
3,002,003
4,003,004


### user table:

In [25]:
user_df = user_raw.loc[:, ["Age", "Gender", "spotify_usage_period", "spotify_subscription_plan",
                          "music_lis_frequency", "fav_music_genre", "music_Influencial_mood"]]
user_df = user_df.dropna()
user_df = user_df.rename(columns = {"spotify_usage_period" : "usage_period",
                                   "spotify_subscription_plan" : "plan",
                                   "music_lis_frequency" : "listening_scenario",
                                   "music_influential_mood" : "mood"})
# Create user_id
user_df["user_id"] = [str(i).zfill(len(str(len(user_df)))) for i in range(len(user_df))]
# Set user password to 000000 for convenience
user_df["password"] = "000000"
user_df.head()

,Age,Gender,usage_period,plan,listening_scenario,fav_music_genre,music_Influencial_mood,user_id,password
0,20-35,Female,More than 2 years,Free (ad-supported),leisure time,Melody,Sadness or melancholy,000,000000
1,12-20,Male,More than 2 years,Free (ad-supported),Workout session,Rap,Social gatherings or parties,001,000000
2,35-60,Others,6 months to 1 year,Free (ad-supported),"Study Hours, While Traveling",Pop,Relaxation and stress relief,002,000000
3,20-35,Female,1 year to 2 years,Free (ad-supported),"Office hours, Workout session, leisure time",Melody,"Relaxation and stress relief, Social gathering...",003,000000
4,20-35,Female,1 year to 2 years,Free (ad-supported),leisure time,Melody,Relaxation and stress relief,004,000000
5,20-35,Male,More than 2 years,Free (ad-supported),Workout session,Pop,Uplifting and motivational,005,000000
6,20-35,Female,1 year to 2 years,Free (ad-supported),Office hours,Melody,Relaxation and stress relief,006,000000
7,20-35,Female,Less than 6 months,Free (ad-supported),leisure time,Melody,Social gatherings or parties,007,000000
8,20-35,Female,Less than 6 months,Free (ad-supported),"While Traveling, leisure time",Melody,"Relaxation and stress relief, Uplifting and mo...",008,000000
9,20-35,Female,More than 2 years,Free (ad-supported),"Office hours, While Traveling, Workout session",Melody,Relaxation and stress relief,009,000000


### matchability table:
The section below shows how to connect the song and user tables from different sources. For each user, we compute a matchability score of each song based on a self-created algorithm. The algorithm involves *fav_music_genre* column from the **user** relation which records each user's favorite music genre and the *danceability_%* and *energy_%* columns from the **song** relation. Finally, based on the standardized matchability score, we create the matchability relation using song_id and user_id, which separately served as foreign keys and together as composite primary key of this relation.

In [51]:
#song recommendation
def song_recommendation(user_id,song_df,user_df):
    
    def get_user_favorite_genre(user_id,user_df):
        user_info = user_df[user_df['user_id'] == user_id]
        if user_info.empty:
            return f"User with ID {user_id} not found."
        favorite_genre = user_info['fav_music_genre'].values[0]
        return favorite_genre



    def rec_songs(min_value, max_value,song_df):
        filtered_songs = song_df[(song_df['criteria'] >= min_value) & (song_df['criteria'] <= max_value)]
        random_5 = filtered_songs.sample(n=5)#, random_state=42)
        selected_song = random_5['song_id'].tolist()
        return selected_song

    
    
    def rec_songs_by_release_date(song_df,date):
        song_df['released_date'] = pd.to_datetime(song_df['released_date'])
        filtered_songs = song_df[song_df['released_date'] >= date]
        random_5 = filtered_songs.sample(n=5)#, random_state=42)
        selected_songs = random_5['song_id'].tolist() 
        return selected_songs

    #all songs
    temp_song_df = song_df.copy()
    temp_song_df['criteria'] = temp_song_df['danceability_%'] + temp_song_df['energy_%']
    temp_song_df['criteria'] = (temp_song_df['criteria'].rank() - 1) / len(temp_song_df) * 100
    
   
    #free songs
    temp_song_df2 = temp_song_df.copy()
    temp_song_df2 = temp_song_df2[temp_song_df2['promotion'] == 1]
    

    
    if (user_id not in user_df['user_id'].values):
        print(f"User {user_id} does not exist.")
        return None
    
    #for free users
    elif (user_df[user_df['user_id'] == user_id]['plan'].values == 'Free (ad-supported)'):
        
        song_list1 = []
        if (get_user_favorite_genre(user_id,user_df)=='classical'):
             song_list1=rec_songs(0,15,temp_song_df2)
        elif (get_user_favorite_genre(user_id,user_df)=='Classical & melody, dance'):
             song_list1=rec_songs(0,50,temp_song_df2)
        elif (get_user_favorite_genre(user_id,user_df)=='Old songs'):
             song_list1=rec_songs(0,30,temp_song_df2)
        elif (get_user_favorite_genre(user_id,user_df)=='Melody'):
             song_list1=rec_songs(25,50,temp_song_df2)
        elif (get_user_favorite_genre(user_id,user_df)=='Pop'):
             song_list1=rec_songs(50,80,temp_song_df2)
        elif (get_user_favorite_genre(user_id,user_df)=='Kpop'):
             song_list1=rec_songs(50,75,temp_song_df2)
        elif (get_user_favorite_genre(user_id,user_df)=='Electronic/Dance'):
             song_list1=rec_songs(50,80,temp_song_df2)
        elif (get_user_favorite_genre(user_id,user_df)=='Rap'):
             song_list1=rec_songs(50,85,temp_song_df2)
        elif (get_user_favorite_genre(user_id,user_df)=='Rock'):
             song_list1=rec_songs(75,100,temp_song_df2)
        elif (get_user_favorite_genre(user_id,user_df)=='trending songs random'):
             song_list1=rec_songs_by_release_date(temp_song_df2, date='2023-01-01')
        else:
             song_list1=rec_songs(0,100,temp_song_df2)
        return  song_list1
        

    #for premium users
    else:
        song_list = []
        if (get_user_favorite_genre(user_id,user_df)=='classical'):
             song_list=rec_songs(0,15,temp_song_df)
        elif (get_user_favorite_genre(user_id,user_df)=='Classical & melody, dance'):
             song_list=rec_songs(0,50,temp_song_df)
        elif (get_user_favorite_genre(user_id,user_df)=='Old songs'):
             song_list=rec_songs(0,30,temp_song_df)
        elif (get_user_favorite_genre(user_id,user_df)=='Melody'):
             song_list=rec_songs(25,50,temp_song_df)
        elif (get_user_favorite_genre(user_id,user_df)=='Pop'):
             song_list=rec_songs(50,80,temp_song_df)
        elif (get_user_favorite_genre(user_id,user_df)=='Kpop'):
             song_list=rec_songs(50,75,temp_song_df)
        elif (get_user_favorite_genre(user_id,user_df)=='Electronic/Dance'):
             song_list=rec_songs(50,80,temp_song_df)
        elif (get_user_favorite_genre(user_id,user_df)=='Rap'):
             song_list=rec_songs(50,85,temp_song_df)
        elif (get_user_favorite_genre(user_id,user_df)=='Rock'):
             song_list=rec_songs(75,100,temp_song_df)
        elif (get_user_favorite_genre(user_id,user_df)=='trending songs random'):
             song_list=rec_songs_by_release_date(temp_song_df, date='2023-01-01')
        else:
             song_list=rec_songs(0,100,temp_song_df)
        return  song_list
   

In [14]:
#test
song_recommendation('500',song_df,user_df)

['Down Under (feat. Colin Hay)',
 "Say You Won't Let Go",
 'Monï¿½ï¿½y so',
 'See You Again',
 'Never Felt So Alone']

In [63]:
matchability = pd.DataFrame()
recommendations = {}
for userid in user_df["user_id"]:
    recommendations[userid] = song_recommendation(userid, song_df, user_df)
matchability = pd.DataFrame(recommendations).T.reset_index()
matchability = matchability.rename(columns = {0:"song_1", 1:"song_2", 2:"song_3", 3:"song_4", 4:"song_5", "index":"user_id"})
matchability.head()

,user_id,song_1,song_2,song_3,song_4,song_5
0,000,353,617,721,567,311
1,001,333,199,016,516,535
2,002,448,338,539,532,208
3,003,472,392,127,592,569
4,004,397,624,430,583,281


In [64]:
# Export to csv
song_df.to_csv("/Users/hangsmacbookair/Desktop/CPSC 537/cpsc537proj/song.csv")
user_df.to_csv("/Users/hangsmacbookair/Desktop/CPSC 537/cpsc537proj/user.csv")
artist_df.to_csv("/Users/hangsmacbookair/Desktop/CPSC 537/cpsc537proj/artist.csv")
song_artists_df.to_csv("/Users/hangsmacbookair/Desktop/CPSC 537/cpsc537proj/song_artist.csv")
matchability.to_csv("/Users/hangsmacbookair/Desktop/CPSC 537/cpsc537proj/matchability.csv")